In [19]:
import os
import random
import pandas as pd
import numpy as np

In [20]:
data_dir = '../data'

In [21]:
df_full = pd.read_csv(os.path.join(data_dir, 'main/cust.csv'))
ids = df_full['id']
df_full['be_segment_p1m'] = df_full['BE_SEGMENT'].apply(lambda x: int(x[0]))
df_full = df_full.iloc[:,-12:].replace({-1:np.nan})
df_full = 1 - (df_full - 1) // 2
df_full.insert(0, 'id', ids)

In [22]:
df_full

,id,be_segment_p12m,be_segment_p11m,be_segment_p10m,be_segment_p9m,be_segment_p8m,be_segment_p7m,be_segment_p6m,be_segment_p5m,be_segment_p4m,be_segment_p3m,be_segment_p2m,be_segment_p1m
0,22348732,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,22355477,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,12359928,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,22364505,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,22366008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,26249148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9996,26250390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9997,26250603,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9998,26252691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df = df_full[['id', 'be_segment_' + pxm, 'be_segment_p1m']]
df.columns = ['id', 't0', 'is_affluent_cur']
df = df[df['t0'] == 0]
df = df.drop_duplicates()

path = os.path.join(data_dir, 'model', pxm)

# split not affluent
random.seed(0)
idx0 = df[df['is_affluent_cur'] == 0].index.tolist()
random.shuffle(idx0)
train0, validate0, test0 = np.split(idx0, [int(len(idx0)*0.7), int(len(idx0)*0.85)])

# split affluent
random.seed(0)
idx1 = df[df['is_affluent_cur'] == 1].index.tolist()
random.shuffle(idx1)
train1, validate1, test1 = np.split(idx1, [int(len(idx1)*0.7), int(len(idx1)*0.85)])

# into dfs
train = df.loc[list(train0) + list(train1)].reset_index(drop = True)[['id', 'is_affluent_cur']]
validate = df.loc[list(validate0) + list(validate1)].reset_index(drop = True).reset_index(drop = True)[['id', 'is_affluent_cur']]
test = df.loc[list(test0) + list(test1)].reset_index(drop = True).reset_index(drop = True)[['id', 'is_affluent_cur']]

train.to_csv(os.path.join(path, 'train.csv'), index = False)
validate.to_csv(os.path.join(path, 'validate.csv'), index = False)
test.to_csv(os.path.join(path, 'test.csv'), index = False)
pd.concat([train, validate, test]).to_csv(os.path.join(path, 'full.csv'), index = False)